<a href="https://colab.research.google.com/github/ferustigris/colab_invest/blob/master/Update_stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

Install dependencies for Yahoo finance and gsheets. As well as client_secret.json required in root directory. Could be downloaded from [here](https://console.cloud.google.com/apis/api/drive.googleapis.com/metrics?project=gold-access-268714)

In [1]:
!pip install pygsheets

In [2]:
!pip install yfinance

# Framework

Create Yahoo API reader:

In [3]:
import yfinance as yf

BLNS = 1000_000_000

class NoValue(Exception):
  pass

class YahooFinanceReader:
    """Yahoo Finance Reader class for access via API"""

    class Ticker:
        """Case-class for the particular ticket"""

        def __init__(self, tkt):
            """Pass ticket object in the constructor"""
            self.tkt = tkt

        def get_current_price(self):
            """Get very current price of the ticket"""
            return self.tkt.info["currentPrice"]

        def get_divs(self):
            """Get annual divs of the ticket"""
            div = None
            try:
              divs = self.tkt.info["fiveYearAvgDividendYield"] / 100
              if not divs:
                try:
                  divs = self.tkt.info["trailingAnnualDividendYield"]
                except KeyError:
                  print("trailingAnnualDividendYield: key error")
              if not divs:
                try:
                  divs = self.tkt.info["dividendYield"]
                except KeyError:
                  print("dividendYield: key error")
              if not divs:
                try:
                  divs = self.tkt.info["forwardAnnualDividendYield"]
                except KeyError:
                  print("forwardAnnualDividendYield: key error")
              return divs
            except KeyError:
                pass
            raise NoValue("ERROR: %s" % "divs key error")

        def get_net(self):
            """Get annual netIncomeToCommon of the ticket in billions"""
            try:
                return self.tkt.info["netIncomeToCommon"] / BLNS
            except KeyError:
                raise NoValue("ERROR: %s" % "netin key error")

        def get_revenue(self):
            try:
                """Get annual totalRevenue of the ticket in billions"""
                return self.tkt.info["totalRevenue"] / BLNS
            except KeyError:
                raise NoValue("ERROR: %s" % "Total revenue key error")

        def get_total_assets(self):
            """Get total assets of the ticket in billions"""
            return self.tkt.info["totalAssets"] / BLNS

        def get_pe(self):
            """Get total assets of the ticket in billions"""
            return self.tkt.info["trailingPE"]

        def get_fpe(self):
            """Get total assets of the ticket in billions"""
            try:
              return self.tkt.info["forwardingPE"]
            except KeyError:
              return self.tkt.info["forwardPE"]

        def get_freecash(self):
            """Get freeCashflow of the ticket in billions"""
            return self.tkt.info["freeCashflow"] / BLNS

        def get_shares(self):
            """Get shares volume of the ticket in billions"""
            impliedSharesOutstanding = self.tkt.info["impliedSharesOutstanding"]
            sharesOutstanding = self.tkt.info["sharesOutstanding"]

            if impliedSharesOutstanding > sharesOutstanding:
              return impliedSharesOutstanding / BLNS
            else:
              return sharesOutstanding / BLNS

        def get_sma200days(self):
            """Get twoHundredDayAverage of the ticket in billions"""
            return self.tkt.info["twoHundredDayAverage"] / BLNS

        def get_history(self):
          return self.tkt.history(period="max")

        def get_debt(self):
            """Get annual total Liabilities of the ticket in billions"""
            try:
                return self.tkt.balance_sheet['Total Liabilities'] / BLNS
            except KeyError:
                pass
            raise NoValue("ERROR: %s" % "Total Liabilities key error")

        def get_assets(self):
           return self.tkt.balance_sheet['Net Tangible Assets'] / BLNS

    def get_ticker(self, ticker_name, cache={}):
        """Returns ticket object by ticket name"""
        if ticker_name in cache.keys():
            return cache[ticker_name]

        ticker = self.Ticker(yf.Ticker(ticker_name))
        cache[ticker_name] = ticker
        print(ticker.tkt.info)
        return ticker


Create gsheet writer, the file [itself](https://docs.google.com/spreadsheets/d/1hc8kcZJDUJ91qL-mp_NDxZUkrOnhihdzNhxVKRF04fQ/edit#gid=1283370971):

In [4]:
import pygsheets

# authorization
MAX_AMOUNT_ROWS = 50
COL_AVG = "" #TODO twoHundredDayAverage
COL_PRICE = 'E'
COL_DIVS = 'V'
COL_REVENUE = 'L'
COL_NET = 'M'
COL_ASSETS = 'O'
COL_SHARES = 'D'
COL_AVG = 'F'
COL_PE = "W"
COL_FPE = "X"
COL_DEBT = "S"
COL_FREECASH = "Y"
NYSE_SHEET_NUMBER = 1
EU_SHEET_NUMBER = 2
PATH_TO_KEY_JSON = 'client_secret.json'
GSHEET_NAME = 'Finance stock and invest'


class GSheetUpdater:
    """Class for Finance stock and invest updates"""
    def __init__(self, sheet=NYSE_SHEET_NUMBER):
        self.__gc = pygsheets.authorize(service_file=PATH_TO_KEY_JSON)
        # open the google spreadsheet (by is the name of my sheet)
        self.__sh = self.__gc.open(GSHEET_NAME)

        # select the right sheet
        self.__wks = self.__sh[sheet]

    def apply_for_all(self, func, col_to_be_updated):
        """applies function for all rows"""
        for i in range(2, MAX_AMOUNT_ROWS):
            header = self.__wks.cell('B%i' % i)
            ticket = header.value
            try:
              self.__wks.update_value('%s%i' % (col_to_be_updated, i), try_func(ticket))
            except NoValue:
              self.__wks.update_value('%s%i' % (col_to_be_updated, i), None)
              print("ERROR: %s" % s)

    def apply_for_all(self, cols_and_funcs, start_from=2):
        """applies function for all rows"""
        for i in range(start_from, MAX_AMOUNT_ROWS):
            header = self.__wks.cell('B%i' % i)
            ticket = header.value
            print("Getting ticket object for %i: %s..." % (i, ticket))
            if not ticket:
              raise NoValue("No ticket found")
            for (col_to_be_updated, func) in cols_and_funcs.items():
              for tries in range(1):
                try:
                  self.__wks.update_value('%s%i' % (col_to_be_updated, i), func(ticket))
                  break
                except NoValue as e:
                    self.__wks.update_value('%s%i' % (col_to_be_updated, i), None)
                    print("ERROR NoValue: %s" % e)
                except KeyError as e:
                    self.__wks.update_value('%s%i' % (col_to_be_updated, i), None)
                    print("ERROR KeyError: %s" % e)



# Update stocks

What we are going to update:

In [5]:
r = YahooFinanceReader()


def update_price(ticket):
    """Apply the function for every ticker"""
    try:
      price = r.get_ticker(ticket).get_current_price()
      return price
    except KeyError as k:
      return None


cols_and_funcs = {
    COL_PRICE: update_price,
    COL_DIVS: lambda ticket: r.get_ticker(ticket).get_divs(),
    COL_REVENUE: lambda ticket: r.get_ticker(ticket).get_revenue(),
    COL_ASSETS: lambda ticket: r.get_ticker(ticket).get_assets(),
    COL_SHARES: lambda ticket: r.get_ticker(ticket).get_shares(),
    COL_NET: lambda ticket: r.get_ticker(ticket).get_net(),
    COL_AVG: lambda ticket: r.get_ticker(ticket).get_sma200days(),
    COL_PE: lambda ticket: r.get_ticker(ticket).get_pe(),
    COL_FPE: lambda ticket: r.get_ticker(ticket).get_fpe(),
    COL_FREECASH: lambda ticket: r.get_ticker(ticket).get_freecash(),
    COL_DEBT: lambda ticket: r.get_ticker(ticket).get_debt()
}


# Update excel

Updating gsheet for EU:

In [6]:
sheet_eu = GSheetUpdater(EU_SHEET_NUMBER)
try:
  sheet_eu.apply_for_all(cols_and_funcs, 2)
except NoValue as e:
  print(e)

Getting ticket object for 2: VOW.DE...
{'address1': 'Berliner Ring 2', 'city': 'Wolfsburg', 'zip': '38440', 'country': 'Germany', 'phone': '49 5361 9 0', 'fax': '49 5361 9 28282', 'website': 'https://www.volkswagen-group.com', 'industry': 'Auto Manufacturers', 'industryKey': 'auto-manufacturers', 'industryDisp': 'Auto Manufacturers', 'sector': 'Consumer Cyclical', 'sectorKey': 'consumer-cyclical', 'sectorDisp': 'Consumer Cyclical', 'longBusinessSummary': 'Volkswagen AG manufactures and sells automobiles in Germany, other European countries, North America, South America, the Asia-Pacific, and internationally. The company operates through four segments: Passenger Cars and Light Commercial Vehicles, Commercial Vehicles, Power Engineering, and Financial Services. The Passenger Cars and Light Commercial Vehicles segment engages in the development of vehicles, engines, and vehicle software; produces and sells passenger cars and light commercial vehicles, and related parts; and offers motorcy

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ORAN?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ORAN&crumb=aS9mPAINySh


{'trailingPegRatio': None}
ERROR NoValue: ERROR: divs key error
ERROR NoValue: ERROR: Total revenue key error
ERROR KeyError: 'Net\xa0Tangible\xa0Assets'
ERROR KeyError: 'impliedSharesOutstanding'
ERROR NoValue: ERROR: netin key error
ERROR KeyError: 'twoHundredDayAverage'
ERROR KeyError: 'trailingPE'
ERROR KeyError: 'forwardPE'
ERROR KeyError: 'freeCashflow'
ERROR NoValue: ERROR: Total Liabilities key error
Getting ticket object for 7: DTE.DE...
{'address1': 'Friedrich-Ebert-Allee 140', 'city': 'Bonn', 'zip': '53113', 'country': 'Germany', 'phone': '49 228 181 88880', 'website': 'https://www.telekom.com', 'industry': 'Telecom Services', 'industryKey': 'telecom-services', 'industryDisp': 'Telecom Services', 'sector': 'Communication Services', 'sectorKey': 'communication-services', 'sectorDisp': 'Communication Services', 'longBusinessSummary': 'Deutsche Telekom AG, together with its subsidiaries, provides integrated telecommunication services. The company operates through Germany, Unite

{'address1': 'Plaza Euskadi número 5', 'city': 'Bilbao', 'zip': '48009', 'country': 'Spain', 'phone': '34 91 784 29 15', 'fax': '34 91 784 36 97', 'website': 'https://www.iberdrola.com', 'industry': 'Utilities - Diversified', 'industryKey': 'utilities-diversified', 'industryDisp': 'Utilities - Diversified', 'sector': 'Utilities', 'sectorKey': 'utilities', 'sectorDisp': 'Utilities', 'longBusinessSummary': 'Iberdrola, S.A. engages in the generation, transmission, distribution, and supply of electricity in Spain, the United Kingdom, the United States, Mexico, Brazil, Germany, France, and Australia. It generates electricity from renewable sources, such as onshore and offshore wind, hydro, photovoltaic, combined cycle gas, and conventional nuclear, as well as through batteries. The company is also involved in the purchase and sale of electricity and gas on wholesale markets; energy retail supply activities, such as gas and electricity, and other products and services, including hydrogen, as

ERROR KeyError: 'Net\xa0Tangible\xa0Assets'
ERROR KeyError: 'forwardPE'
ERROR KeyError: 'freeCashflow'
ERROR NoValue: ERROR: Total Liabilities key error
Getting ticket object for 9: SWR...
{'exchange': 'YHD', 'quoteType': 'MUTUALFUND', 'symbol': 'SWR', 'underlyingSymbol': 'SWR', 'firstTradeDateEpochUtc': 1405344600, 'timeZoneFullName': 'America/New_York', 'timeZoneShortName': 'EST', 'uuid': 'fc571139-fd81-34da-bc35-57d6dbe4cbd4', 'gmtOffSetMilliseconds': -18000000, 'maxAge': 86400, 'trailingPegRatio': None}
ERROR NoValue: ERROR: divs key error
ERROR NoValue: ERROR: Total revenue key error
ERROR KeyError: 'Net\xa0Tangible\xa0Assets'
ERROR KeyError: 'impliedSharesOutstanding'
ERROR NoValue: ERROR: netin key error
ERROR KeyError: 'twoHundredDayAverage'
ERROR KeyError: 'trailingPE'
ERROR KeyError: 'forwardPE'
ERROR KeyError: 'freeCashflow'
ERROR NoValue: ERROR: Total Liabilities key error
Getting ticket object for 10: DPW.DE...


...

{'trailingPegRatio': None}
ERROR NoValue: ERROR: divs key error
ERROR NoValue: ERROR: Total revenue key error
ERROR KeyError: 'Net\xa0Tangible\xa0Assets'
ERROR KeyError: 'impliedSharesOutstanding'
ERROR NoValue: ERROR: netin key error
ERROR KeyError: 'twoHundredDayAverage'
ERROR KeyError: 'trailingPE'
ERROR KeyError: 'forwardPE'
ERROR KeyError: 'freeCashflow'
ERROR NoValue: ERROR: Total Liabilities key error
Getting ticket object for 11: IMPP...
{'address1': '331 Kifissias Avenue', 'address2': 'Erithrea', 'city': 'Athens', 'zip': '14561', 'country': 'Greece', 'phone': '30 210 62 50 001', 'website': 'https://www.imperialpetro.com', 'industry': 'Oil & Gas Midstream', 'industryKey': 'oil-gas-midstream', 'industryDisp': 'Oil & Gas Midstream', 'sector': 'Energy', 'sectorKey': 'energy', 'sectorDisp': 'Energy', 'longBusinessSummary': 'Imperial Petroleum Inc. provides international seaborne transportation services to oil producers, refineries, and commodities traders. It carries refined petrol

ERROR NoValue: ERROR: Total Liabilities key error
Getting ticket object for 14: E...
{'address1': 'Piazzale Enrico Mattei 1', 'city': 'Rome', 'state': 'RM', 'zip': '00144', 'country': 'Italy', 'phone': '39 06 59821', 'fax': '39 06 59 82 21 41', 'website': 'https://www.eni.com', 'industry': 'Oil & Gas Integrated', 'industryKey': 'oil-gas-integrated', 'industryDisp': 'Oil & Gas Integrated', 'sector': 'Energy', 'sectorKey': 'energy', 'sectorDisp': 'Energy', 'longBusinessSummary': 'Eni S.p.A. operates as an integrated energy company worldwide. The company engages in exploration, development, extracting, manufacturing, and marketing crude oil and natural gas, oil-based fuels, chemical products, and gas-fired power, as well as energy products from renewable sources. It operates through Exploration & Production; Global Gas & LNG Portfolio (GGP); Enilive, Refining and Chemicals; Plenitude & Power; and Corporate and Other Activities segments. The company engages in research, development, and pr

ERROR KeyError: 'Net\xa0Tangible\xa0Assets'
ERROR NoValue: ERROR: Total Liabilities key error
Getting ticket object for 20: SNW.F...
{'address1': '46, avenue de la Grande Armée,', 'city': 'Paris', 'zip': '75017', 'country': 'France', 'phone': '33 1 53 77 40 00', 'website': 'https://www.sanofi.com', 'industry': 'Drug Manufacturers - General', 'industryKey': 'drug-manufacturers-general', 'industryDisp': 'Drug Manufacturers - General', 'sector': 'Healthcare', 'sectorKey': 'healthcare', 'sectorDisp': 'Healthcare', 'longBusinessSummary': 'Sanofi, a healthcare company, engages in the research, development, manufacture, and marketing of therapeutic solutions in the United States, Europe, Canada, and internationally. It operates through Pharmaceuticals, Vaccines, and Consumer Healthcare segments. The company provides specialty care, such as DUPIXENT, neurology and immunology, rare diseases, oncology, and rare blood disorders; medicines for diabetes and cardiovascular diseases; and established 

ERROR NoValue: ERROR: Total Liabilities key error
Getting ticket object for 25: BBVA...
{'address1': 'Plaza San NicolAs, 4', 'city': 'Bilbao', 'zip': '48005', 'country': 'Spain', 'website': 'https://www.bbva.com', 'industry': 'Banks - Diversified', 'industryKey': 'banks-diversified', 'industryDisp': 'Banks - Diversified', 'sector': 'Financial Services', 'sectorKey': 'financial-services', 'sectorDisp': 'Financial Services', 'longBusinessSummary': 'Banco Bilbao Vizcaya Argentaria, S.A. provides retail banking, wholesale banking, and asset management services in the United States, Spain, Mexico, Turkey, South America, and internationally. The company offers savings account, demand deposits, and time deposits; and loan products, such as residential mortgages, other households, credit card loans, loans to enterprises and public sector, as well as consumer finance. It provides insurance and asset management business, including corporate, commercial, SME, payment systems, retail, private and 

ERROR KeyError: 'Net\xa0Tangible\xa0Assets'
ERROR NoValue: ERROR: Total Liabilities key error
Getting ticket object for 32: ASML...
{'address1': 'De Run 6501', 'city': 'Veldhoven', 'zip': '5504 DR', 'country': 'Netherlands', 'phone': '31 48 0235 0934', 'website': 'https://www.asml.com', 'industry': 'Semiconductor Equipment & Materials', 'industryKey': 'semiconductor-equipment-materials', 'industryDisp': 'Semiconductor Equipment & Materials', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'ASML Holding N.V. develops, produces, markets, sells, and services advanced semiconductor equipment systems for chipmakers. It offers advanced semiconductor equipment systems, including lithography, metrology, and inspection systems. The company also provides extreme ultraviolet lithography systems; and deep ultraviolet lithography systems comprising immersion and dry lithography solutions to manufacture various range of semiconductor nodes and te

Updating gsheet for NYSE:

In [7]:
sheet_ny = GSheetUpdater(NYSE_SHEET_NUMBER)
try:
  sheet_ny.apply_for_all(cols_and_funcs, 2)
except NoValue as e:
  print(e)

Getting ticket object for 2: RIO...
{'address1': '6 St James’s Square', 'city': 'London', 'zip': 'SW1Y 4AD', 'country': 'United Kingdom', 'phone': '44 20 7781 2000', 'website': 'https://www.riotinto.com', 'industry': 'Other Industrial Metals & Mining', 'industryKey': 'other-industrial-metals-mining', 'industryDisp': 'Other Industrial Metals & Mining', 'sector': 'Basic Materials', 'sectorKey': 'basic-materials', 'sectorDisp': 'Basic Materials', 'longBusinessSummary': 'Rio Tinto Group engages in exploring, mining, and processing mineral resources worldwide. The company operates through Iron Ore, Aluminium, Copper, and Minerals Segments. The Iron Ore segment engages in the iron ore mining, and salt and gypsum production in Western Australia. The Aluminum segment is involved in bauxite mining; alumina refining; and aluminium smelting. The Copper segment engages in mining and refining of copper, gold, silver, molybdenum, and other by-products and exploration activities. The Minerals segment

ERROR KeyError: 'Net\xa0Tangible\xa0Assets'
ERROR NoValue: ERROR: Total Liabilities key error
Getting ticket object for 6: OKE...
{'address1': '100 West Fifth Street', 'city': 'Tulsa', 'state': 'OK', 'zip': '74103', 'country': 'United States', 'phone': '918 588 7000', 'website': 'https://www.oneok.com/', 'industry': 'Oil & Gas Midstream', 'industryKey': 'oil-gas-midstream', 'industryDisp': 'Oil & Gas Midstream', 'sector': 'Energy', 'sectorKey': 'energy', 'sectorDisp': 'Energy', 'longBusinessSummary': 'ONEOK, Inc. engages in gathering, processing, fractionation, storage, transportation, and marketing of natural gas and natural gas liquids (NGL) in the United States. It operates through four segments: Natural Gas Gathering and Processing, Natural Gas Liquids, Natural Gas Pipelines, and Refined Products and Crude. The company owns natural gas gathering pipelines and processing plants in the Mid-Continent and Rocky Mountain regions; and provides midstream services to producers of NGLs. It 

ERROR KeyError: 'freeCashflow'
ERROR NoValue: ERROR: Total Liabilities key error
Getting ticket object for 12: KHC...
{'address1': 'One PPG Place', 'city': 'Pittsburgh', 'state': 'PA', 'zip': '15222', 'country': 'United States', 'phone': '412 456 5700', 'website': 'https://www.kraftheinzcompany.com', 'industry': 'Packaged Foods', 'industryKey': 'packaged-foods', 'industryDisp': 'Packaged Foods', 'sector': 'Consumer Defensive', 'sectorKey': 'consumer-defensive', 'sectorDisp': 'Consumer Defensive', 'longBusinessSummary': "The Kraft Heinz Company, together with its subsidiaries, manufactures and markets food and beverage products in North America and internationally. Its products include condiments and sauces, cheese and dairy products, meals, meats, refreshment beverages, coffee, and other grocery products under the Kraft, Oscar Mayer, Heinz, Philadelphia, Lunchables, Velveeta, Ore-Ida, Maxwell House, Kool-Aid, Jell-O, Heinz, ABC, Master, Quero, Kraft, Golden Circle, Wattie's, Pudliszki,

ERROR NoValue: ERROR: divs key error
ERROR KeyError: 'Net\xa0Tangible\xa0Assets'
ERROR NoValue: ERROR: Total Liabilities key error
Getting ticket object for 19: GAMB...
{'address1': '22 Grenville Street', 'city': 'Saint Helier', 'zip': 'JE4 8PX', 'country': 'Jersey', 'website': 'https://www.gambling.com/corporate', 'industry': 'Gambling', 'industryKey': 'gambling', 'industryDisp': 'Gambling', 'sector': 'Consumer Cyclical', 'sectorKey': 'consumer-cyclical', 'sectorDisp': 'Consumer Cyclical', 'longBusinessSummary': "Gambling.com Group Limited operates as a performance marketing company for the online gambling industry worldwide. It provides digital marketing services for the iGaming and social casino products. The company's focus is on online casino, online sports betting, and fantasy sports industry. It publishes various branded websites, including Gambling.com, Casinos.com, RotoWire.com, and Bookies.com. The company was incorporated in 2006 and is based in Saint Helier, Jersey.", 'full

ERROR NoValue: ERROR: Total Liabilities key error
Getting ticket object for 24: MSFT...
{'address1': 'One Microsoft Way', 'city': 'Redmond', 'state': 'WA', 'zip': '98052-6399', 'country': 'United States', 'phone': '425 882 8080', 'website': 'https://www.microsoft.com', 'industry': 'Software - Infrastructure', 'industryKey': 'software-infrastructure', 'industryDisp': 'Software - Infrastructure', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, including Dynamics 

ERROR NoValue: ERROR: Total Liabilities key error
Getting ticket object for 30: DAL...
{'address1': 'PO Box 20706', 'city': 'Atlanta', 'state': 'GA', 'zip': '30320-6001', 'country': 'United States', 'phone': '404 715 2600', 'website': 'https://www.delta.com', 'industry': 'Airlines', 'industryKey': 'airlines', 'industryDisp': 'Airlines', 'sector': 'Industrials', 'sectorKey': 'industrials', 'sectorDisp': 'Industrials', 'longBusinessSummary': 'Delta Air Lines, Inc. provides scheduled air transportation for passengers and cargo in the United States and internationally. The company operates through two segments, Airline and Refinery. Its domestic network centered on core hubs in Atlanta, Minneapolis-St. Paul, Detroit, and Salt Lake City, as well as coastal hub positions in Boston, Los Angeles, New York-LaGuardia, New York-JFK, and Seattle; and international network centered on hubs and market presence in Amsterdam, Bogota, Lima, Mexico City, London-Heathrow, Paris-Charles de Gaulle, Sao Pau

ERROR NoValue: ERROR: divs key error
ERROR KeyError: 'Net\xa0Tangible\xa0Assets'
ERROR NoValue: ERROR: Total Liabilities key error
Getting ticket object for 36: PYPL...
{'address1': '2211 North First Street', 'city': 'San Jose', 'state': 'CA', 'zip': '95131', 'country': 'United States', 'phone': '408 967 1000', 'website': 'https://www.paypal.com', 'industry': 'Credit Services', 'industryKey': 'credit-services', 'industryDisp': 'Credit Services', 'sector': 'Financial Services', 'sectorKey': 'financial-services', 'sectorDisp': 'Financial Services', 'longBusinessSummary': 'PayPal Holdings, Inc. operates a technology platform that enables digital payments on behalf of merchants and consumers worldwide. It operates a two-sided network at scale that connects merchants and consumers that enables its customers to connect, transact, and send and receive payments through online and in person, as well as transfer and withdraw funds using various funding sources, such as bank accounts, PayPal or V

ERROR NoValue: ERROR: Total Liabilities key error
Getting ticket object for 41: HUBB...
{'address1': '40 Waterview Drive', 'city': 'Shelton', 'state': 'CT', 'zip': '06484-1000', 'country': 'United States', 'phone': '475 882 4000', 'website': 'https://www.hubbell.com', 'industry': 'Electrical Equipment & Parts', 'industryKey': 'electrical-equipment-parts', 'industryDisp': 'Electrical Equipment & Parts', 'sector': 'Industrials', 'sectorKey': 'industrials', 'sectorDisp': 'Industrials', 'longBusinessSummary': 'Hubbell Incorporated, together with its subsidiaries, designs, manufactures, and sells electrical and utility solutions in the United States and internationally. It operates through two segments, Electrical Solutions and Utility Solutions. The Electrical Solution segment offers standard and special application wiring device products, rough-in electrical products, connector and grounding products, lighting fixtures, and other electrical equipment for use in industrial, commercial, and